<a href="https://colab.research.google.com/github/Sshettyyy/Minor_Project/blob/main/Suhas1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
import os
import base64

In [3]:
pip install pillow

In [4]:
pip install pycryptodome

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 11.0 MB/s eta 0:00:00


In [5]:
from PIL import Image
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes

In [6]:
def encrypt_aes(message, key):
    cipher = AES.new(key, AES.MODE_EAX)
    nonce = cipher.nonce
    ciphertext, tag = cipher.encrypt_and_digest(message)
    return nonce + ciphertext + tag

In [7]:
def decrypt_aes(encrypted_data, key):
    nonce = encrypted_data[:16]
    ciphertext = encrypted_data[16:-16]
    tag = encrypted_data[-16:]
    cipher = AES.new(key, AES.MODE_EAX, nonce=nonce)
    decrypted_data = cipher.decrypt_and_verify(ciphertext, tag)
    return decrypted_data

In [8]:
DELIMITER = "11111111"

def embed_message(image_path, file_path, secret_message):
    img = Image.open(image_path)
    width, height = img.size
    if len(secret_message) + len(DELIMITER) + os.path.getsize(file_path) * 8 > width * height * 3:
        raise ValueError("Message is too large to embed in the image.")

    # Embed the secret message
    binary_message = ''.join(format(ord(char), '08b') for char in secret_message)
    binary_message += DELIMITER
    data = img.getdata()
    new_data = []
    message_index = 0

    for pixel in data:
        if message_index < len(binary_message):
            r, g, b = pixel
            r = r & ~1 | int(binary_message[message_index])
            message_index += 1
            new_data.append((r, g, b))
        else:
            new_data.append(pixel)

    # Save the modified image
    new_img = Image.new(img.mode, img.size)
    new_img.putdata(new_data)
    new_img.save("stego_image.png")
    print("Message embedded successfully!")

In [9]:
DELIMITER = "11111111"

# Function to retrieve the encoded file path and secret message from the stego image
def retrieve_message(stego_image_path):
    img = Image.open(stego_image_path)
    data = img.getdata()
    binary_message = ""
    delimiter_found = False

    for pixel in data:
        r, _, _ = pixel
        binary_message += str(r & 1)
        if binary_message.endswith(DELIMITER):
            delimiter_found = True
            break

    if not delimiter_found:
        raise ValueError("Delimiter not found in the stego image.")

    # Remove the delimiter and convert binary to text
    secret_message = ''.join(chr(int(binary_message[i:i + 8], 2)) for i in range(0, len(binary_message) - len(DELIMITER), 8))

    return secret_message.strip()

In [10]:
if __name__ == "__main__":
    # Get the secret message from the user
    secret_message = input("Enter the secret message: ").encode()  # Encode the string into bytes

    # AES Encryption
    aes_key = get_random_bytes(32)  # 256-bit key
    encrypted_data = encrypt_aes(secret_message, aes_key)

    # Store the encrypted data in a file
    encrypted_file_path = "encrypted_data.bin"
    with open(encrypted_file_path, "wb") as f:
        f.write(encrypted_data)

    # Encode the file path using Base64
    encoded_file_path = base64.urlsafe_b64encode(encrypted_file_path.encode()).decode()

    # Embed the file path and secret message in the cover image
    cover_image_path = "/content/gdrive/MyDrive/cc.jpg"  # Replace with the path to your cover image
    embed_message(cover_image_path, encrypted_file_path, encoded_file_path + DELIMITER + secret_message.decode())

    # Retrieving the encoded data from the stego image
    stego_image_path = "stego_image.png"  # Replace with the path to the stego image
    extracted_data = retrieve_message(stego_image_path)

    # Split the extracted data into the encoded file path and the secret message
    extracted_encoded_file_path, extracted_secret_message = extracted_data.split(DELIMITER)

    # Decode the file path and secret message from Base64
    decoded_file_path = base64.urlsafe_b64decode(extracted_encoded_file_path).decode()

    # Check if the file exists before attempting to open it
    if os.path.exists(decoded_file_path):
        # Decrypt the extracted encrypted data
        with open(decoded_file_path, "rb") as f:
            extracted_encrypted_data = f.read()
            decrypted_message = decrypt_aes(extracted_encrypted_data, aes_key)

        print("Decrypted Message:", decrypted_message.decode())
    else:
        print("Error: The file does not exist.")

Enter the secret message: hi
Message embedded successfully!
Decrypted Message: hi


In [11]:
import numpy as np
import math
from PIL import Image

In [12]:
def calculate_mse(original, stego):
    mse = np.mean((original - stego)**2)
    return mse

In [13]:
def calculate_psnr(max_pixel_value, mse):
    if mse == 0:
        return float('inf')  # Perfect match, PSNR is infinity
    psnr = 10 * math.log10((max_pixel_value ** 2) / mse)
    return psnr

In [14]:
# Load cover and stego images
cover_image_path = '/content/gdrive/MyDrive/cc.jpg'
stego_image_path = '/content/stego_image.png'

cover_image = np.array(Image.open(cover_image_path))
stego_image = np.array(Image.open(stego_image_path))

# Constants
MAX_PIXEL_VALUE = 255  # Assuming 8-bit images

# Calculate MSE and PSNR
mse = calculate_mse(cover_image, stego_image)
psnr = calculate_psnr(MAX_PIXEL_VALUE, mse)

# Print the results
print(f"MSE: {mse}")
print(f"PSNR: {psnr}")

MSE: 0.00015159841579655492
PSNR: 86.32385697925498


In [15]:
from PIL import Image

# Open the image
image_path = "/content/gdrive/MyDrive/cc.jpg"  # Replace with the actual path to your image
image = Image.open(image_path)

# Get the dimensions (size) of the image
width, height = image.size

# Print the dimensions
print("Image Width:", width)
print("Image Height:", height)


Image Width: 678
Image Height: 467
